In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import random
import numpy as np
import pandas as pd
from random import randint

In [3]:
ingredients_df = pd.read_csv('/content/drive/Shareddrives/genetic algorithm/codes/ingredients-dataset.csv')
df = ingredients_df.dropna()

In [4]:
DOMAIN = len(df)-1
LEN_CHROMOSOME = 15

In [5]:
def create_individual(N=DOMAIN, len_chromosome=LEN_CHROMOSOME):
    return [randint(0,N) for _ in range(len_chromosome)]

In [6]:
def get_population(n_population):
  population = [create_individual() for _ in range(n_population)]
  return population

In [7]:
def get_needs(age, weight, height, f_activity=1.7):
  amb = float(66.5 + (13.7 * weight) + (5 * height) - (6.8 * age))
  tee = float(amb * f_activity)

  # because these are still ingredients that need to be cooked,
  # we spare some space for the other ingredients
  const = 0.95 
  protein = float(0.25 * tee * const)
  fat = float(0.15 * tee * const)
  carb = float(0.60 * tee * const)

  return protein, fat, carb

In [8]:
def get_nutrition_and_price(individual):
  sum_price = sum_protein = sum_carb = sum_fat = 0

  for allele in individual:
    sum_protein += df.iloc[allele]['Protein(kal)']
    sum_fat += df.iloc[allele]['Lemak(kal)']
    sum_carb += df.iloc[allele]['Karbohidrat(kal)']
    sum_price +=  df.iloc[allele]['Harga (IDR)']

  return [sum_protein, sum_fat, sum_carb], sum_price

def get_fitness(individual, needs, alpha_protein=5, alpha_fat=5, alpha_carb=5):
  nutrition, price = get_nutrition_and_price(individual)
  protein, fat, carb = needs
  penalty_protein = float(abs(nutrition[0] - protein))
  penalty_fat = float(abs(nutrition[1] - fat))
  penalty_carb = float(abs(nutrition[2] - carb))
  
  sum_penalties = (alpha_protein*penalty_protein) + (alpha_fat*penalty_fat) + (alpha_carb*penalty_carb)

  fitness = 1/((price/1000) + sum_penalties)
  return fitness

In [9]:
def selection(population, fitness, n_tournament=3):
  selection_index = 0
  for n in range(n_tournament):
    r = randint(0, len(population)-1)
    if fitness[r] > fitness[selection_index]:
      selection_index = r
  return population[selection_index]

In [10]:
def crossover(parent1, parent2, pc):
  if random.uniform(0.0, 1.0) < pc:
    point1 = randint(1, len(parent1)-3)
    point2 = randint(point1, len(parent1)-2)

    children1 = parent2[:point1] + parent1[point1:point2] + parent2[point2:]
    children2 = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
    return children1, children2
  else:
    return parent1, parent2

In [11]:
def mutation(individual, pm):
  for allele in range(len(individual)):
    if random.uniform(0.0, 1.0) < pm:
      creep = randint(-3, 3)
      individual[allele] = individual[allele] + creep
      # change with random numbers if the mutation was outside the domain
      if (individual[allele]<0 or individual[allele]>DOMAIN):
        individual[allele] = randint(0,DOMAIN)
  return individual

In [12]:
def continuous_update(prev_population, children, n_population):
  all_population = prev_population + children
  
  selected_idx = set()
  while len(selected_idx) < n_population:
    idx = randint(0, len(all_population)-1)
    selected_idx.add(idx)
  
  new_population = list()
  for idx in selected_idx:
    new_population.append(all_population[idx])

  assert len(new_population) == n_population
  return new_population

In [13]:
def genetic_algorithm(needs, n_iter, n_population, pc, pm):
  population = get_population(n_population)
  
  best_fits = list()

  best_individual, best_fit = population[0], get_fitness(population[0], needs)
  best_fits.append(best_fit)
  
  for generation in range(n_iter):
    fitness = [get_fitness(individual, needs) for individual in population]
    for individual in range(n_population):
      if fitness[individual] > best_fit:
        best_individual, best_fit = population[individual], fitness[individual]
        best_fits.append(best_fit)
    print(f'{generation+1}-th, best fit: {population[individual]}, fitness: {fitness[individual]}')
    
    mating_pool = n_population//2
    if (mating_pool%2 is not 0):
      mating_pool += 1

    parents = [selection(population, fitness) for _ in range(mating_pool)]
    children = list()
    for i in range(0, mating_pool, 2):
      parent1, parent2 = parents[i], parents[i+1]
      for individual in crossover(parent1, parent2, pc):
        mutation(individual, pm)
        children.append(individual)
    population = continuous_update(population, children, n_population)

  best_fitness_mean = sum(best_fits)/len(best_fits)
  return best_individual, best_fit, best_fitness_mean

In [14]:
def exhaustive_search(needs):
  n_population_params = [10, 30, 50]
  n_iteration = 100
  pc_params = [0.8, 0.85, 0.9]
  pm_params = [0.0002, 0.067]

  best_n_population = n_population_params[0]
  best_pc = pc_params[0]
  best_pm = pm_params[0]

  best_individual, best_fit, best_fitness_mean = genetic_algorithm(needs, n_iteration, best_n_population, best_pc, best_pm)

  for n_pop_param in n_population_params:
      for pc_param in pc_params:
        for pm_param in pm_params:
            individual, fit, fitness_mean = genetic_algorithm(needs, n_iteration, n_pop_param, pc_param, pm_param)
            if fit > best_fit:
              best_individual = individual

              best_fit = fit
              best_fitness_mean = fitness_mean
              best_n_population = n_pop_param
              best_pc = pc_param
              best_pm = pm_param

  return best_individual, best_fit, best_fitness_mean, best_n_population, n_iteration, best_pc, best_pm


In [15]:
age = 23
weight = 70
height = 175
activity = "ringan"    
ACTIVITY_MAP = {'istirahat_total': 1,
             'sangat_ringan': 1.3,
             'ringan': 1.6,
             'sedang': 1.7,
             'berat': 2.1,
             'sangat_berat': 2.4}

In [16]:
needs = get_needs(age, weight, height, ACTIVITY_MAP[activity])

In [17]:
best_individual, best_fit, best_fitness_mean, best_n_population, best_n_iter, best_pc, best_pm = exhaustive_search(needs)

1-th, best fit: [56, 152, 163, 68, 119, 66, 183, 163, 75, 54, 152, 114, 35, 109, 30], fitness: 0.0003656093245002123
2-th, best fit: [56, 152, 163, 68, 119, 66, 183, 163, 141, 188, 20, 86, 35, 109, 30], fitness: 0.00032288884777437573
3-th, best fit: [87, 27, 176, 118, 195, 131, 84, 14, 81, 180, 131, 62, 9, 155, 5], fitness: 0.00033040997600232354
4-th, best fit: [23, 183, 145, 68, 119, 8, 68, 57, 119, 66, 20, 86, 69, 183, 26], fitness: 0.0002932327740406891
5-th, best fit: [23, 183, 145, 78, 99, 8, 68, 57, 119, 66, 20, 86, 69, 183, 26], fitness: 0.0005405405405405412
6-th, best fit: [23, 183, 145, 78, 99, 8, 68, 57, 119, 66, 20, 86, 69, 183, 26], fitness: 0.0005405405405405412
7-th, best fit: [23, 183, 145, 78, 99, 8, 68, 57, 119, 66, 20, 86, 69, 183, 26], fitness: 0.0005405405405405412
8-th, best fit: [23, 183, 145, 78, 99, 8, 68, 57, 119, 66, 20, 86, 69, 183, 26], fitness: 0.0005405405405405412
9-th, best fit: [23, 183, 145, 78, 99, 8, 68, 57, 119, 66, 20, 86, 69, 183, 26], fitness:

In [18]:
print(f'got best parameter n_population: {best_n_population}, n_iteration: {best_n_iter}, best_pc: {best_pc}, best_pm: {best_pm}')

got best parameter n_population: 50, n_iteration: 100, best_pc: 0.85, best_pm: 0.067


In [19]:
print(f'best individual: {best_individual}, with fitness: {best_fit}')

best individual: [35, 24, 130, 95, 110, 63, 45, 44, 99, 136, 69, 17, 60, 190, 139], with fitness: 0.00814100215736563


In [20]:
ingredients = df.iloc[best_individual]
ingredients

,No,Bahan,Satuan (gr),Protein(gr),Lemak(gr),Karbohidrat(gr),Protein(kal),Lemak(kal),Karbohidrat(kal),Harga (IDR),total kalori
35,35,Pisang raja,130,1.2,0.2,31.8,4.8,1.8,127.2,3900,133.8
24,24,Kacang Merah Rebus (Segar),100,10.3,0.9,28.2,41.2,8.1,112.8,3900,162.1
130,130,Hofa/Ubi hutan,100,1.2,0.5,29.5,4.8,4.5,118.0,2300,127.3
95,95,Arrowroot,100,1.0,0.2,24.1,4.0,1.8,96.4,4500,102.2
110,110,Jeruk Bali,100,0.6,0.2,12.4,2.4,1.8,49.6,7300,53.8
63,63,Kacang gude,100,20.7,1.0,58.0,82.8,9.0,232.0,2800,323.8
45,45,Ikan cakalang,100,19.6,0.7,5.5,78.4,6.3,22.0,2800,106.7
44,44,Kacang arab,100,23.8,1.4,60.2,95.2,12.6,240.8,2850,348.6
99,99,Kacang uci,100,23.4,2.4,60.6,93.6,21.6,242.4,10000,357.6
136,136,Rusip,100,11.5,2.0,11.7,46.0,18.0,46.8,6000,110.8


In [21]:
protein, fat, carb = needs
print(f'protein: {protein}, fat: {fat}, carbohidrat:{carb}')

protein: 662.7579999999999, fat: 397.65479999999997, carbohidrat:1590.6191999999999


In [22]:
ingredients['Karbohidrat(kal)'].sum()

1573.6

In [23]:
ingredients['Lemak(kal)'].sum()

226.79999999999998

In [24]:
ingredients['Protein(kal)'].sum()

563.6000000000001

In [25]:
ingredients['Harga (IDR)'].sum()

69350